In [1]:
import pickle
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

from timeseries_point import extract_series

In [ ]:
DATE = "20210626"
files = Path(f"../data/{DATE}").glob("*.grib2")
files = sorted(files)

In [3]:
usgs_data_path = Path("usgs/usgs_data.pkl")

with open(usgs_data_path, "rb") as f:
    data = pickle.load(f)

In [4]:
daily, df, site_info, pcodes = data.values()

site_info

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no,geometry
0,USGS,413104087440001,"RAIN GAGE AT MATTESON, IL",AT,413104,874400,41.517778,-87.733333,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAWS0,POINT (-87.73333 41.51778)
1,USGS,413510087380201,"RAIN GAGE AT HARVEY, IL",AT,413510,873802,41.586111,-87.633889,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAWS00,POINT (-87.63389 41.58611)
2,USGS,413516087442101,"RAIN GAGE AT OAK FOREST, IL",AT,413516,874421,41.587778,-87.739167,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAWS00,POINT (-87.73917 41.58778)
3,USGS,413548087395901,RAIN GAGE AT CALUMET UNION DR CANAL NR MARKHAM...,AT,413548,873959,41.596667,-87.666389,M,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001UM,POINT (-87.66639 41.59667)
4,USGS,414028087383501,"RAIN GAGE AT WEST PULLMAN AT CHICAGO, IL",AT,414028,873835,41.674444,-87.643056,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001CC,POINT (-87.64306 41.67444)
5,USGS,414030087450001,"RAIN GAGE AT ALSIP, IL",AT,414030,874500,41.675000,-87.750000,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAWS00,POINT (-87.75 41.675)
6,USGS,415755087525300,"OHARE AIRPORT AT CHICAGO, IL",AT,415755,875253,41.965308,-87.881453,M,S,...,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,11303,POINT (-87.88145 41.96531)


In [ ]:
from collections import namedtuple

NamedCoord = namedtuple("NamedCoord", ["name", "lat", "lon"])

coords = {}

for i, site in site_info.iterrows():
    coord = NamedCoord(
        site["station_nm"],
        site["dec_lat_va"],
        360 + site["dec_long_va"],
    )

    df = extract_series(files, coord.lat, coord.lon)
    dest_folder = Path(f"./timeseries/{DATE}")
    df.to_csv(dest_folder / f"{coord.name}.csv")